In [359]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

In [360]:
rid = defaultdict(list)

In [361]:
# <경기도 버스노선 조회 API>로부터 운행지역별 노선번호 목록 조회
for i in [13, 2, 11, 15, 12, 28, 14, 25, 27, 17]: # 등록 장애인 수 기준 상위 10개 도시의 지역 id
    if i < 10:
        areabusroute = 'http://apis.data.go.kr/6410000/busrouteservice/getAreaBusRouteList?serviceKey=k9sTEVvsgPYra2RkrXzQSnQX0BROQsLNMk1yPFZEId7c76sfg8y8jsrg1oRRzaSt%2B4Gk7ijz0T%2BVcp0hDKNy3A%3D%3D&areaId=0'+str(i)
    elif i >= 10:
        areabusroute = 'http://apis.data.go.kr/6410000/busrouteservice/getAreaBusRouteList?serviceKey=k9sTEVvsgPYra2RkrXzQSnQX0BROQsLNMk1yPFZEId7c76sfg8y8jsrg1oRRzaSt%2B4Gk7ijz0T%2BVcp0hDKNy3A%3D%3D&areaId='+str(i)   
    response = requests.get(areabusroute)
    routeList = response.content
    fileName = "/home/ukjung18/project/busRouteInfo/AreaBusRoute"+str(i)+".xml"
    with open(fileName, "wb") as file:
        file.write(routeList)
    fp = open("/home/ukjung18/project/busRouteInfo/AreaBusRoute"+str(i)+".xml","r")
    soup = BeautifulSoup(fp, "html.parser")
    for route in soup.findAll('busroutelist'):
        if (route.routetypename.string == '일반형시내버스') | (route.routetypename.string == '일반형농어촌버스') | (route.routetypename.string == '마을버스'):
            rid[str(i)].append(route.routeid.string)

In [362]:
len(rid['13'])

49

In [363]:
import pandas as pd
routeId = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in rid.items()]))

In [364]:
# routeIdcpt : 등록 장애인 수 기준 상위 10개 도시
routeId.to_csv("routeIdcpt.csv", mode='w', index=False)

In [377]:
# <경기도 버스노선 조회 API>로부터 노선별 경유정류소 목록 조회
for i in [13, 2, 11, 15, 12, 28, 14, 25, 27, 17]:
    for j in rid[str(i)]:
        x=[]
        y=[]
        br = 'http://apis.data.go.kr/6410000/busrouteservice/getBusRouteStationList?serviceKey=k9sTEVvsgPYra2RkrXzQSnQX0BROQsLNMk1yPFZEId7c76sfg8y8jsrg1oRRzaSt%2B4Gk7ijz0T%2BVcp0hDKNy3A%3D%3D&areaId=0&routeId='+j
        response = requests.get(br)
        route = response.content
        fileName1 = "/home/ukjung18/project/busRoute/"+str(i)+"/"+j+".xml"
        with open(fileName1, "wb") as file:
            file.write(route)
        fp1 = open("/home/ukjung18/project/busRoute/"+str(i)+"/"+j+".xml","r")
        soup = BeautifulSoup(fp1, "html.parser")
        for stop in soup.findAll('busroutestationlist'):
            x.append(stop.x.string)
            y.append(stop.y.string)
        busLine = pd.DataFrame()
        busLine['x'] = x
        busLine['y'] = y
        busLine.to_csv("/home/ukjung18/project/busRoute/"+str(i)+"/"+j+".csv", mode='w', index=False)